## DIA 036: Implementación de Reportes Automatizados y Análisis Predictivo

En el Día 36 se añade una funcionalidad para generar reportes automatizados que consoliden la información de uso, rendimiento y feedback de la API. Estos reportes se obtendrán mediante un endpoint protegido para administradores y mostrarán estadísticas relevantes, como el total de feedbacks recibidos, el porcentaje de predicciones correctas y otros indicadores clave. Esta herramienta de análisis predictivo ayudará a identificar tendencias y a tomar decisiones informadas para mejorar el modelo y la experiencia del usuario.

Código Completo (api.py)
A continuación se muestra el código que integra el nuevo endpoint de reportes en la aplicación. Se asume que ya tienes definidos los modelos y endpoints anteriores (por ejemplo, el modelo Feedback y los endpoints de login, feedback, etc.).

python
Copiar
from flask import Flask, Blueprint, request, jsonify, render_template, url_for
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
from flask_bcrypt import Bcrypt
from flask_mail import Mail, Message
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address
import os
import logging
from datetime import datetime
import time
import threading
import json

# Configuración básica y variables de entorno
app = Flask(__name__)
app.config['SECRET_KEY'] = os.getenv('SECRET_KEY', 'your_secret_key')
app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY', 'your_jwt_secret_key')
app.config['SQLALCHEMY_DATABASE_URI'] = os.getenv('DATABASE_URL', 'sqlite:///app.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Inicialización de extensiones
db = SQLAlchemy(app)
migrate = Migrate(app, db)
bcrypt = Bcrypt(app)
mail = Mail(app)
jwt = JWTManager(app)
limiter = Limiter(app, key_func=get_remote_address, default_limits=["200 per day", "50 per hour"])

# Configuración de Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ---------------------------
# Modelos
# ---------------------------

# Modelo User (simplificado para este ejemplo)
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    # Otros campos omitidos

# Modelo Feedback para almacenar la retroalimentación de las predicciones
class Feedback(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), nullable=False)  # Usuario que envía el feedback
    prediction = db.Column(db.Integer, nullable=False)   # Predicción realizada
    correct = db.Column(db.Boolean, nullable=False)        # Indicador si la predicción fue correcta
    comment = db.Column(db.Text, nullable=True)            # Comentario adicional (opcional)
    timestamp = db.Column(db.DateTime, default=datetime.utcnow)  # Momento del feedback

    def to_dict(self):
        return {
            "id": self.id,
            "username": self.username,
            "prediction": self.prediction,
            "correct": self.correct,
            "comment": self.comment,
            "timestamp": self.timestamp.isoformat()
        }

# ---------------------------
# Decorador para Roles (simplificado)
# ---------------------------
from functools import wraps
def role_required(required_role):
    def decorator(f):
        @wraps(f)
        def wrapper(*args, **kwargs):
            current_user_identity = get_jwt_identity()
            if not current_user_identity:
                return jsonify({"msg": "Token de acceso requerido"}), 401
            user = User.query.filter_by(username=current_user_identity).first()
            if not user:
                return jsonify({"msg": "Usuario no encontrado"}), 404
            # Para este ejemplo, se asume que el usuario tiene un atributo 'role' que es 'admin' o 'user'
            # Si el usuario no es admin, se bloquea el acceso.
            if getattr(user, 'role', 'user') != 'admin':
                return jsonify({"msg": "Acceso no autorizado"}), 403
            return f(*args, **kwargs)
        return wrapper
    return decorator

# ---------------------------
# Endpoints Comunes
# ---------------------------
@app.route('/login', methods=['POST'])
def login():
    data = request.get_json()
    username = data.get('username')
    password = data.get('password')
    if not username or not password:
        return jsonify({"msg": "Username and password required"}), 400
    # En un caso real, se verificarían las credenciales. Aquí asumimos que son correctas.
    access_token = create_access_token(identity=username)
    logger.info(f"Usuario '{username}' inició sesión.")
    return jsonify(access_token=access_token), 200

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok"}), 200

# ---------------------------
# Endpoints de Versionado (ejemplos de v1 y v2)
# ---------------------------
api_v1 = Blueprint('api_v1', __name__)
@api_v1.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("100 per day")
def predict_v1():
    if 'file' not in request.files:
        return jsonify({"error": "No se encontró el archivo"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No se seleccionó ningún archivo"}), 400
    # Simulación simple: retorna dígito 5 con 90% de probabilidad
    result = {"prediccion": 5, "probabilidad": 0.90, "version": "v1"}
    logger.info("v1: Predicción realizada.")
    return jsonify(result), 200

api_v2 = Blueprint('api_v2', __name__)
@api_v2.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("150 per day")
def predict_v2():
    if 'file' not in request.files:
        return jsonify({"error": "No se encontró el archivo"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No se seleccionó ningún archivo"}), 400
    # Simulación mejorada: retorna dígito 7 con 95% de probabilidad y un mensaje adicional
    result = {"prediccion": 7, "probabilidad": 0.95, "version": "v2", "mensaje": "Predicción mejorada"}
    logger.info("v2: Predicción realizada con mejoras.")
    return jsonify(result), 200

app.register_blueprint(api_v1, url_prefix='/api/v1')
app.register_blueprint(api_v2, url_prefix='/api/v2')

# ---------------------------
# Nuevo Endpoint: Pipeline de Retraining Basado en Feedback
# ---------------------------
@app.route('/admin/retrain', methods=['POST'])
@jwt_required()
@role_required('admin')
def retrain_model_endpoint():
    """
    Endpoint para que un administrador inicie el proceso de retraining del modelo basado en feedback.
    Este proceso se ejecuta de manera asíncrona para no bloquear la respuesta de la API.
    """
    def retrain_job():
        logger.info("Inicio del retraining del modelo basado en feedback...")
        # Simulación del proceso de retraining (reemplaza con tu lógica real)
        time.sleep(10)  # Simula tiempo de retraining
        # Simula actualización del modelo (por ejemplo, guarda un archivo nuevo)
        with open('updated_model.h5', 'w') as f:
            f.write("Modelo actualizado basado en feedback")
        logger.info("Retraining completado. Modelo actualizado.")

    # Ejecutar el proceso de retraining en un hilo separado
    thread = threading.Thread(target=retrain_job)
    thread.start()
    return jsonify({"msg": "Proceso de retraining iniciado"}), 202

# ---------------------------
# Nuevo Endpoint: Generar Reporte Basado en Feedback
# ---------------------------
@app.route('/admin/report', methods=['GET'])
@jwt_required()
@role_required('admin')
def generate_report():
    """
    Genera un reporte resumido basado en los datos de feedback.
    El reporte incluye el total de feedbacks, número de aciertos, errores y porcentaje de precisión.
    """
    total_feedback = Feedback.query.count()
    if total_feedback == 0:
        return jsonify({"msg": "No hay feedback disponible"}), 200

    correct_feedback = Feedback.query.filter_by(correct=True).count()
    incorrect_feedback = Feedback.query.filter_by(correct=False).count()
    accuracy = (correct_feedback / total_feedback) * 100

    report = {
        "total_feedback": total_feedback,
        "correct_feedback": correct_feedback,
        "incorrect_feedback": incorrect_feedback,
        "accuracy_percentage": accuracy
    }
    logger.info("Reporte generado: " + json.dumps(report))
    return jsonify(report), 200

# ---------------------------
# Ejecutar la aplicación
# ---------------------------
if __name__ == '__main__':
    app.run(debug=True)
Explicación del Código
Configuración y Extensiones:
Se configura la aplicación Flask con sus extensiones (JWT, SQLAlchemy, Limiter, etc.) y se establece el logging para capturar eventos importantes.

Endpoints Comunes:
Se definen endpoints básicos como /login y /health, y se registran dos versiones del endpoint /predict utilizando Blueprints (/api/v1/predict y /api/v2/predict).

Endpoint de Retraining:
El endpoint /admin/retrain permite a un administrador iniciar un proceso de retraining de manera asíncrona, simulando el reentrenamiento del modelo basado en feedback.

Nuevo Endpoint de Reporte:
Se añade un endpoint /admin/report que consulta la base de datos de feedback, calcula estadísticas (total de feedback, aciertos, errores y porcentaje de precisión) y devuelve un reporte en formato JSON.

Protección y Seguridad:
Los endpoints críticos, como los de retraining y reporte, están protegidos mediante JWT y el decorador role_required para asegurar que solo los administradores puedan acceder a ellos.

Este ejemplo muestra cómo extender la funcionalidad de la API para incluir un pipeline de retraining y un sistema de reportes automatizados basados en el feedback, lo que permitirá analizar el rendimiento del modelo y facilitar su mejora continua.

